# Detecting 1-3rd degree relationships with READ2
This script uses pseudo-haploid genotype calls as an input and READ2 method to detect 1-3rd degree relationships among the individuals. 

Resources: https://github.com/GuntherLab/READv2


This all below ran in 7 mins
replace the N-s in the ANGSD output with 0-s as a PLINK code for missing data in tped format

sed -i.bak 's/N/0/g' angsd_output.tped

In [ ]:
#!/bin/bash
#SBATCH -J READ2
#SBATCH --time=1:00:00
#SBATCH --ntasks=1
#SBATCH --mem=5g
#SBATCH -A ealloc_e7679_project1-tk-echo
#SBATCH --nodes=1
#SBATCH --output=READ2.txt

module load any/plink/1.9
module load readv2/2.00

# Detecting 1-3rd degree relationships with READ2
# This script uses pseudo-haploid genotype calls as an input and READ2 method to detect 1-3rd degree relationships among the individuals. 
# https://github.com/GuntherLab/READv2
# This all below ran in 7 mins
# replace the N-s in the ANGSD output with 0-s as a PLINK code for missing data in tped format
# sed -i.bak 's/N/0/g' angsd_output.tped

# As READ2 takes PLINK .bed files as input convert the .tped to .bed with PLINK
plink --tfile angsd_output --make-bed --out g

# Prepare .fam file with information from bamlist (otherwise the ANGSD output is ind0, ind1...)
sed -e 's#/gpfs/helios/projects/echo_workshops/project.1.tk/data/Gretzinger/##g' -e 's#.A0101.TF1.1.bam##g' -e 's#.A0101.SG1.2.bam##g' -e 's#.A0201.TF1.1.bam##g' -e 's#.SG1.2.bam##g' -e 's#.B0101.TF1.1.bam##g' -e 's#.TF1.1.bam##g' -e 's#.bam##g' /gpfs/helios/projects/echo_workshops/project.1.tk/data/ADN_HAD.bams >ids.txt

# make a new .fam file with sample IDs
mv g.fam g.ofam
paste ids.txt ids.txt g.ofam|sed -e 's/ /\t/g'|cut -f1,2,5-8 >g.fam

# run READ2 kinship analysis
READ2 -i g

# extract result lines with P0<0.96 that should contain the cases of closest relatedness (1-3)
awk 'BEGIN {FS="\t"}; $5<0.96' Read_Results.tsv|cut -f1,2,5,8 >READ_output_1_3.txt
